# Computational Economics and Nonlinear Modeling (2)


## : General Introduction

PSE Summer School, 2017

Pablo Winant

# Julia Packages
The goal of this notebook is to quickly introduce Julia packages, useful to modelers.

## Packages

Huge ecosystem: http://pkg.julialang.org/pulse.html

Based on Git/Github:

- packages are repositories
- a package = module + files + scripts

Three types of packages:

- shipped (in [JuliaPro](http://juliacomputing.com/products/juliapro.html)) 
- registered (see [METADATA.jl](https://github.com/JuliaLang/METADATA.jl)): install with `Pkg.add()`
- unregistered: install with  `Pkg.clone()

Using a package:
- `using Dolo`: imports all the exported functions
- `import Dolo`: needs qualified access (`Dolo.time_iteration`)

## A small list of useful packages:

*nonlinear optimization / minimization*

- Roots
- Optim
- NLSolve (*)

*interpolation*

- Interpolations (*)
- BasisMatrices (*)
- splines.jl (**)

*Statistics*

- DataFrames
- Distributions
- GLM
- MultivariateStats (*)

*Plotting libraries*

- Gadfly
- PyPlot
- Plots (*)
- Plotly / PlotlyJS (*)

*Formal calculation*

- Calculus
- SymEngine
- ForwardDiff (*)

*Modeling*
- Dolang (**)
- Dolo (**)
- Dyno (**)

See [ucidatascienceinitiative](http://ucidatascienceinitiative.github.io/IntroToJulia/) for a more data-oriented introduction.

## Juno

Atom-based Julia IDE

- atom features: 

    - command palette (ctrl+shift+p), multiple cursors, plugins
    
- juno features:

    - enable workspace (default layout)
    - julia console (command line integration)
    - progressbar
    - debugger

### Exercise 1: basic package management

- list all installed packages
- find directory of installed packages
- inspect content of package QuantEcon
- install (or clone) Dolo.jl

## Topic: statistics

- Arrays and DataFrames:
    - past and present: DataFrames
    - future:???
        - database oriented: DataTables ( [JuliaData](https://github.com/JuliaData) )
        - labelled arrays: AxisArrays or NamedArrays (see [JuliaArrays](https://github.com/JuliaArrays?type=)
- Satistics (see [JuliaStats](https://github.com/JuliaStats) )
    - Distributions.jl
    - StatsBase.jl
    - RDatasets.jl
    
- Econometrics
    - TimeSeries.jl
    - GLM.jl
    - Multivariate.jl

### Exercise 2: a (tiny) bit of statistics

- Create a simple random DataFrame with the following columns (N=100 observations):

    - x and y random, z = 0.5*x + 0.5 y
    - $x$ and $y$ correlated (use Distributions.jl)

- Compute descriptive statistics

(for those who have a good connection):

- Load the LifeCycleSavings database from RData using RDatasets.jl.
    
    - use GLM.jl to regress SR on Pop15, Pop75, DPI, DDPI using OLS

#### Solution


## Topic: Plotting


- PyPlot
    - (transparent wrapper to Python's Matplotlib)
    - very mature, publication-grade quality, even better with 2.0
    - matlab-like interface
- Gadfly
    - native Julia
    - grammar of graphics approach
- Plotly/PlotlyJS
    - very complete API
    - produces interactive Javascript
- Plots
    - simple to use higher level api

### Exercise 3: plotting

Using PyPlot and/or Gadfly:

- plot $f: x\rightarrow \frac{sin(\epsilon + \frac{a |x|}{\pi})}{\epsilon + \frac{a |x|}{\pi}}$ for different values of a on the same graph (with labels, titles, ...)
- plot a contour plot for $g: x, y\rightarrow \frac{sin(\frac{\sqrt{x^2+y^2}}{\pi})}{\frac{\sqrt{x^2+y^2}}{\pi}}$

- (optional): use Plotly or Plots
- (optional 2):  make a 3d graph for $g(x,y)$ using Plotly

#### Solution

## Topic: Maximization/optimization

Two big organizations:

- JuliaOpt: many wrappers, being streamlined towards mathematical programming
- JuliaNLsolve:
    - Optim: maximization
    - NLSolve: nonlinear solution
- MathJulia:
    - Roots: simple api for 1d functions

Remark: the need to jit-compile repeated optimization call partly explains the non-intuitive interfaces.

### Exercise 3: maximization with Optim.jl

- Use Optim to find the maximum of g, with initial point $x_0=[0.1, 0.1]$
- Use Optim to find the maximum of g, with initial point $x_0=[1.1, 5.1]$

#### Solution

### Exercise 4: solving a nonlinear function with Rootls.jl and NLsolve.jl

1. First, we would like to find the smallest root $x_0$ such that $f(x_0)=0$ using Roots.jl
  - find the root $x_0$
  - start from different initial points

2. Now we look for some values $(x_0,y_0)$ such that $g(x_0, y_0)=0$ and $y_0=x_0^2$
  - Define the function which computes the residuals
  - Find a (many solutions) using NLsolve

#### Solution

Let's solve a 1-dimensional function using Roots.

Now let's solve $g$ using a nonlinear solver.

## Topic: Manipulating equations / models


- Differentiating equations:

  - calculus.jl
  - symengine.jl (faster, c++ dependencies)
  
- Automatic differentiation (see [JuliaDiff](https://github.com/JuliaDiff):
  
  - forwardiff.jl

- Dynamic/economic equations

  - dolang
      - equations with timing
      - metaprogramming
      - syntactic checks for model files

### Calculus.jl

We can also use Calculus.jl to perform symbolic differentiation.

In [24]:
import Calculus
Calculus.differentiate("sin(epsilon + sqrt(x^2+y^2))/(epsilon + sqrt(x^2+y^2))", [:x,:y])

2-element Array{Any,1}:
 :(((((2x) * (0.5 / sqrt(x ^ 2 + y ^ 2))) * cos(epsilon + sqrt(x ^ 2 + y ^ 2))) * (epsilon + sqrt(x ^ 2 + y ^ 2)) - sin(epsilon + sqrt(x ^ 2 + y ^ 2)) * ((2x) * (0.5 / sqrt(x ^ 2 + y ^ 2)))) / (epsilon + sqrt(x ^ 2 + y ^ 2)) ^ 2)
 :(((((2y) * (0.5 / sqrt(x ^ 2 + y ^ 2))) * cos(epsilon + sqrt(x ^ 2 + y ^ 2))) * (epsilon + sqrt(x ^ 2 + y ^ 2)) - sin(epsilon + sqrt(x ^ 2 + y ^ 2)) * ((2y) * (0.5 / sqrt(x ^ 2 + y ^ 2)))) / (epsilon + sqrt(x ^ 2 + y ^ 2)) ^ 2)

In [25]:
function residual!(z, res)
    epsilon = 0.0001
    x = z[1]
    y = z[2]
    res[1] = sin(epsilon + sqrt(x^2+y^2))/(epsilon + sqrt(x^2+y^2))
    res[2] = (y - x^2)
    return res
end
# we copy/paste:
function jacobian!(z, res)
    epsilon = 0.0001
#     res = zeros(2,2)
    x = z[1]
    y = z[2]
    res[1,1] = (((((2x) * (0.5 / sqrt(x ^ 2 + y ^ 2))) * cos(epsilon + sqrt(x ^ 2 + y ^ 2))) * (epsilon + sqrt(x ^ 2 + y ^ 2)) - sin(epsilon + sqrt(x ^ 2 + y ^ 2)) * ((2x) * (0.5 / sqrt(x ^ 2 + y ^ 2)))) / (epsilon + sqrt(x ^ 2 + y ^ 2)) ^ 2)
    res[1,2] = (((((2y) * (0.5 / sqrt(x ^ 2 + y ^ 2))) * cos(epsilon + sqrt(x ^ 2 + y ^ 2))) * (epsilon + sqrt(x ^ 2 + y ^ 2)) - sin(epsilon + sqrt(x ^ 2 + y ^ 2)) * ((2y) * (0.5 / sqrt(x ^ 2 + y ^ 2)))) / (epsilon + sqrt(x ^ 2 + y ^ 2)) ^ 2)
    res[2,1] = -2.0*x
    res[2,2] = 1.0
end

jacobian! (generic function with 1 method)

In [26]:
sol = NLsolve.nlsolve( NLsolve.DifferentiableMultivariateFunction(residual!, jacobian!), [+0.5, 0.0])

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.5, 0.0]
 * Zero: [1.63739, 2.68103]
 * Inf-norm of residuals: 0.000000
 * Iterations: 6
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 7
 * Jacobian Calls (df/dx): 7

### Dolang.jl

Functions can also be created/differentiated with [Dolang.jl](https://github.com/EconForge/Dolang)

In [1]:
import Dolang
using Dolang
import Base

In [2]:
expr = parse("sin(epsilon + sqrt(x^2+y^2))/(epsilon + sqrt(x^2+y^2))")
expr2 = parse("y-x^2")

latex_string = string("\$", Dolang.latex(expr), "\$")
Base.HTML(latex_string)

HTML{String}("\$\\frac{\\text{sin}\\left(\\epsilon+\\text{sqrt}\\left(\\left(x\\right)^{2}+\\left(y\\right)^{2}\\right)\\right)}{\\epsilon+\\text{sqrt}\\left(\\left(x\\right)^{2}+\\left(y\\right)^{2}\\right)}\$")

In [3]:
ff = Dolang.FunctionFactory([expr, expr2] , [:x,:y], [:epsilon]; funname=:gg)
code = Dolang.make_function(ff)
gg, gg! = eval(code)

(gg, gg!)

In [4]:
gg([0.0,0.1], [0.00001])

2-element Array{Float64,1}:
 0.998334
 0.1     

In [5]:
gg(Der{1},[0.0,0.1], [0.00001])

2×2 Array{Float64,2}:
  0.0  -0.0333033
 -0.0   1.0      